In [1]:
import bs4
import pandas as pd
import requests

In [2]:
res = requests.get("https://www.flashscore.mobi/match/f3eqDO5s/?t=match-statistics")

In [3]:
soup = bs4.BeautifulSoup(res.text)

### Teams

In [4]:
teams = [t.strip().lower() for t in soup.find("h3").text.split('-')]
assert len(teams) == 2

In [5]:
teams

['germany', 'greece']

### Datetime

In [6]:
pd.to_datetime(soup.find_all(class_="detail")[-1].text, dayfirst=True)

Timestamp('2024-06-07 20:45:00')

### Goals

In [7]:
goals = [int(g) for g in soup.find(class_="detail").find('b').text.split(':')]

In [8]:
goals

[2, 1]

### Statistics

In [9]:
def parse_statistics_row(row: bs4.BeautifulSoup) -> tuple[str, tuple[float, float]]:
    values = [col.text for col in row.find_next().children]
    assert len(values) == 3    
    first_value = float(values[0].removesuffix('%'))
    name = values[1].lower().removesuffix('(xg)').strip().replace(' ', '_')
    second_value = float(values[2].removesuffix('%'))
    return name, (first_value, second_value)

In [10]:
statistics_rows = soup.find(id="statistics-mobi").find_all("div", class_="statisticsMobi")
statistics = dict(map(parse_statistics_row, statistics_rows))

In [11]:
statistics

{'expected_goals': (0.78, 2.21),
 'ball_possession': (64.0, 36.0),
 'goal_attempts': (15.0, 12.0),
 'shots_on_goal': (7.0, 6.0),
 'shots_off_goal': (6.0, 2.0),
 'blocked_shots': (2.0, 4.0),
 'free_kicks': (15.0, 14.0),
 'corner_kicks': (13.0, 3.0),
 'offsides': (3.0, 2.0),
 'goalkeeper_saves': (5.0, 5.0),
 'fouls': (11.0, 13.0),
 'yellow_cards': (0.0, 1.0),
 'total_passes': (678.0, 297.0),
 'completed_passes': (616.0, 237.0),
 'tackles': (5.0, 13.0),
 'attacks': (131.0, 66.0),
 'dangerous_attacks': (59.0, 34.0),
 'crosses_completed': (5.0, 3.0)}

### DataFrame

In [12]:
df = pd.DataFrame(statistics, index=teams)
df['goals'] = goals
df

,expected_goals,ball_possession,goal_attempts,shots_on_goal,shots_off_goal,blocked_shots,free_kicks,corner_kicks,offsides,goalkeeper_saves,fouls,yellow_cards,total_passes,completed_passes,tackles,attacks,dangerous_attacks,crosses_completed,goals
germany,0.78,64.0,15.0,7.0,6.0,2.0,15.0,13.0,3.0,5.0,11.0,0.0,678.0,616.0,5.0,131.0,59.0,5.0,2
greece,2.21,36.0,12.0,6.0,2.0,4.0,14.0,3.0,2.0,5.0,13.0,1.0,297.0,237.0,13.0,66.0,34.0,3.0,1
